# Boosting (XGBoost, LightGBM, CatBoost)

## Gradient Boosting Algorithms

<img src='https://cdn-images-1.medium.com/max/2000/1*i0CA9ho0WArOj-0UdpuKGQ.png' aligh='center'>

The common ensemble techniques like random forests rely on simple averaging of models in the ensemble. The family of boosting methods is based on a different, constructive strategy of ensemble formation. The main idea of boosting is to add new models to the ensemble sequentially. <br>

In gradient boosting  the learning procedure consecutively fits new models to provide a more accurate estimate of the response variable. The principle idea behind this algorithm is to construct the new base-learners to be maximally correlated with the negative gradient of the loss function, associated with the whole ensemble. Simply put, the next algorithm tries to fix the error of the previous one.

### Friedman's classic GBM algorithm

So, now we can finally define classic GBM algorithm, suggested by Jerome Friedman in 1999. It is superved algorithm that has the following components:

- dataset $\large \left\{ (x_i, y_i) \right\}_{i=1, \ldots,n}$;
- number of iterations $\large M$;
- choice of loss function $\large L(y, f)$ with defined gradient;
- choice of function family of base algorithms $\large h(x, \theta)$, with the procedure of its training;
- Additional hyperparameters $\large h(x, \theta)$, for example, tree's depth of decision trees;

The only thing left is the initial approximation $\large f_0(x)$. For simplicity, for an initial approximation a constant value $\large \gamma$ is used. The constant value, as well as optimal coefficient $\large \rho $, are retrieved by binary search or by another line search algorithm regarding initial loss function (not a gradient). So, GBM algorithm:

1. Initialize GBM with constant value $\large \hat{f}(x) = \hat{f}_0, \hat{f}_0 = \gamma,  \gamma \in \mathbb{R}$
$\large \hat{f}_0 = \underset{\gamma}{\arg\min} \ \sum_{i = 1}^{n} L(y_i, \gamma)$
2. For each iteration $\large t = 1, \dots, M$ repeat:
1. Calculate pseudo-residuals $\large r_t$
$\large r_{it} = -\left[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f(x)=\hat{f}(x)}, \quad \mbox{for } i=1,\ldots,n$
2. Build new base algorithm $\large h_t(x)$ as regression on pseudo-residuals $\large \left\{ (x_i, r_{it}) \right\}_{i=1, \ldots,n}$
3. Find optimal coefficient $\large \rho_t $ at $\large h_t(x)$ regarding initial loss function
$\large \rho_t = \underset{\rho}{\arg\min} \ \sum_{i = 1}^{n} L(y_i, \hat{f}(x_i) +  \rho \cdot h(x_i, \theta))$
4. Save $\large \hat{f_t}(x) = \rho_t \cdot h_t(x)$
5. Update current approximation $\large \hat{f}(x)$
$\large \hat{f}(x) \leftarrow \hat{f}(x) + \hat{f_t}(x) = \sum_{i = 0}^{t} \hat{f_i}(x)$
3. Compose final GBM model $\large \hat{f}(x)$
$\large \hat{f}(x) = \sum_{i = 0}^M \hat{f_i}(x) $
4. Conquer kaggle and world by trained model (make predictions, you can handle it by yorself)


### Step-by-step example of how GBM works

Let's try on a toy example to understand how GBM works. With this example we will restore a noisy function $\large y = cos(x) + \epsilon, \epsilon \sim \mathcal{N}(0, \frac{1}{5}), x \in [-5,5]$.

<img src='https://habrastorage.org/web/9fe/04d/7ba/9fe04d7ba5a645d49fc6aa3e875c8c41.jpg'   align='center'>

It's a regression problem that has target variable as a real value, that is why we will use the mean squared error loss function. We will generate 300 pairs of observations, and we will approximate with decision trees of depth 2. Let's put together everything we need to use GBM:
- Toy data $\large \left\{ (x_i, y_i) \right\}_{i=1, \ldots,300}$ ✓
- Number of iterations $\large M = 3$ ✓;
- The mean squared error loss function $\large L(y, f) = (y-f)^2$ ✓
- Gradient $\large L(y, f) = L_2$ loss is just residuals of $\large r = (y - f)$ ✓;
- Decision trees as base algorithms $\large h(x)$ ✓;
- Hyperparameters of the decision trees: trees depth is equal to 2 ✓;

For the mean squared error, initialization $\large \gamma$ and coefficients $\large \rho_t$ are simple. We initialize GBM with the average value $\large \gamma = \frac{1}{n} \cdot \sum_{i = 1}^n y_i$, and all $\large \rho_t$ are equal to 1.

We will run GBM and draw two types of graphs: current approximation $\large \hat{f}(x)$ (blue graph), and also every tree $\large \hat{f_t}(x)$ built on their pseudo-residuals (green graph). The graph's number corresponds to the iteration's number:

<img src='https://habrastorage.org/web/edb/328/98a/edb32898ad014d8d95782759d11f63fb.png'   align='center'>

Note that by the second iteration our trees repeated the basic form of the function. However, at the first iteration we see that the algorithm built only the "left branch" of the function ($\large x \in [-5, -4]$). It happend due to the fact that our trees simply did not have enough depth to build a symmetrical branch at once, and the error on the left brach was bigger. So the right branch appeared only on the second iteration.

The rest of the process went as we expected: on every step our pseudo-residuals decresed, as a results, GBM better and better with each iteration approximated the original function. However, trees by construction cannot approximate a continuous function, therefore on this example GBM is useful, but not ideal. To play for yourself how GBM approximates functions, in the blog [Brilliantly wrong](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html) there are awesome interactive demo:

<img src='https://habrastorage.org/web/779/3e0/e66/7793e0e66b7d4871b6391a94cd5d4cf2.jpg'   align='center'>[http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)


### Boosting parameters
Tunning parameters is very important stage, we can significantly improve our baseline model. It is very difficult to get answers to practical questions like – "Which set of parameters should be tuned?" Let's take a look on general GBM parameters

All GBM parameters can be divided in three main groups:
1. Tree-Specific Parameters;
2. Boosting Parameters;
3. Miscellaneous Parameters.


The optimal parameters of a model can depend on many scenarios. So it is impossible to create a comprehensive guide for doing so.

# Xgboost

Its name stands for eXtreme Gradient Boosting, it was developed by Tianqi Chen and now is part of a wider collection of open-source libraries developed by the Distributed Machine Learning Community (DMLC). XGBoost is a scalable and accurate implementation of gradient boosting machines and it has proven to push the limits of computing power for boosted trees algorithms as it was built and developed for the sole purpose of model performance and computational speed. Specifically, it was engineered to exploit every bit of memory and hardware resources for tree boosting algorithms.

The implementation of XGBoost offers several advanced features for model tuning, computing environments and algorithm enhancement. It is capable of performing the three main forms of gradient boosting (Gradient Boosting (GB), Stochastic GB and Regularized GB) and it is robust enough to support fine tuning and addition of regularization parameters. According to Tianqi Chen, the latter is what makes it superior and different to other libraries.

### Objective function

$$ \large obj^{(t)}=\sum_{i=1}^{n}[l(y_i,ŷ_{i}^{(t−1)})+g_{i}f_{t}(x_i)+\frac{1}{2}h_{i}f_{t}^{2}(x_{i})]+\Omega(f_t)+constant $$

where $\large g_{i}$ and $\large h_{i}$ – are first and second derivatives.

### Model Complexity

$$ \large \Omega(f) = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T}(w_j^2)$$

where $\large w$ – vector of scores on leaves; $\large T$is the number of leaves; $\large \gamma$ and $\large \lambda$ – regularization parameters.


### Boosting settings

**Explanation after lightgbm theory**<br>
`tree_method` – 'exact' (if you have time, you can try), 'approx', 'hist'(the best choise usually) <br>
`grow_policy` – 'depthwise', 'lossguide' (usually better)<br>
`objective` – default is quite good (sometimes "count:poisson")


### XGBoost parameters tuning

Usually we start tuning parameters with these first: <br>
`n_estimators` - number of boosting rounds, better to use `early_stopping_rounds` <br>
`eta` – learning rate, increasing lr reduces convergence time. (usually default value works good)

### Control Overfitting
When you observe high training accuracy, but low tests accuracy, it is likely that you encounter overfitting problem.

There are in general two ways that you can control overfitting:

* The first way is to directly control model complexity <p>
`max_depth` - maximum depth of a tree, increase of this value will make the model more complex; <br>
`gamma` - minimum loss reduction required to make a further partition on a leaf node of the tree.<br>
`min_child_weight` – minimum sum of instance weight (hessian) needed in a child.


* The second way is to add randomness to make training robust to noise <p>
`subsample` - subsample ratio of the training instance, <br>
`colsample_bytree` - subsample ratio of columns when constructing each tree. <br>


### Handle Imbalanced Dataset
There are two ways to improve it:

* If you care only about the ranking order (AUC) of your prediction
Balance the positive and negative weights, via `scale_pos_weight`
Use AUC for evaluation
* If you care about predicting the right probability
In such a case, you cannot re-balance the dataset. Set parameter `max_delta_step` to a finite number (say 1) will help to converge <br>

More about xgboost parameters: https://xgboost.readthedocs.io/en/latest/parameter.html

Always use `early_stopping_round` and tune `n_estimators` on validation.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold

data = pd.read_csv('/content/drive/MyDrive/projector_course_data/credit_scoring_sample.csv', sep=";")

y = data["SeriousDlqin2yrs"].astype(int)
X = data.drop(["SeriousDlqin2yrs"], axis=1)
X = X.fillna(X.median())

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [3]:
y.value_counts()

0    35037
1    10026
Name: SeriousDlqin2yrs, dtype: int64

In [4]:
import xgboost as xgb


parameters = {
    #default
    "objective": "binary:logistic",
    "eta": 0.1,
    "verbosity": 0,
    "nthread": 4,
    "random_seed": 1,
    "eval_metric": "auc"
}


xgb_train = xgb.DMatrix(X, y, feature_names=list(X.columns))

In [5]:
results = xgb.cv(parameters, xgb_train, num_boost_round=100,
                 folds=skf, verbose_eval=10)

[0]	train-auc:0.82687+0.00325	test-auc:0.82153+0.00298
[10]	train-auc:0.83529+0.00184	test-auc:0.82831+0.00254
[20]	train-auc:0.84113+0.00161	test-auc:0.83154+0.00245
[30]	train-auc:0.84646+0.00127	test-auc:0.83399+0.00241
[40]	train-auc:0.85086+0.00156	test-auc:0.83526+0.00228
[50]	train-auc:0.85369+0.00152	test-auc:0.83562+0.00216
[60]	train-auc:0.85638+0.00151	test-auc:0.83564+0.00220
[70]	train-auc:0.85845+0.00198	test-auc:0.83559+0.00224
[80]	train-auc:0.86044+0.00182	test-auc:0.83542+0.00216
[90]	train-auc:0.86305+0.00193	test-auc:0.83515+0.00222
[99]	train-auc:0.86483+0.00253	test-auc:0.83492+0.00228


In [6]:
# added early stopping
num_rounds = 10000
results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=10)

[0]	train-auc:0.82687+0.00325	test-auc:0.82153+0.00298
[10]	train-auc:0.83529+0.00184	test-auc:0.82831+0.00254
[20]	train-auc:0.84113+0.00161	test-auc:0.83154+0.00245
[30]	train-auc:0.84646+0.00127	test-auc:0.83399+0.00241
[40]	train-auc:0.85086+0.00156	test-auc:0.83526+0.00228
[50]	train-auc:0.85369+0.00152	test-auc:0.83562+0.00216
[60]	train-auc:0.85638+0.00151	test-auc:0.83564+0.00220
[64]	train-auc:0.85734+0.00176	test-auc:0.83559+0.00223


In [7]:
%%time
parameters = {
    #default
    "objective": "binary:logistic",
    "eta": 0.1,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": 1,
    "eval_metric": "auc",

    # regularization parameters
    "max_depth": 5,
    "subsample": 0.7,
    "colsample_bytree": 0.7
}

results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=10)

[0]	train-auc:0.79231+0.00339	test-auc:0.79117+0.00668
[10]	train-auc:0.83662+0.00124	test-auc:0.83188+0.00309
[20]	train-auc:0.84091+0.00106	test-auc:0.83395+0.00242
[30]	train-auc:0.84285+0.00110	test-auc:0.83475+0.00223
[40]	train-auc:0.84403+0.00111	test-auc:0.83482+0.00224
[50]	train-auc:0.84596+0.00129	test-auc:0.83530+0.00243
[60]	train-auc:0.84757+0.00140	test-auc:0.83568+0.00245
[70]	train-auc:0.84930+0.00155	test-auc:0.83569+0.00248
[73]	train-auc:0.84978+0.00154	test-auc:0.83566+0.00240
CPU times: user 4.64 s, sys: 644 ms, total: 5.28 s
Wall time: 3.32 s


### We will be back after LightGBM theory

In [8]:
%%time
parameters = {
    #default
    "objective": "binary:logistic",
    "eta": 0.1,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": 1,
    "eval_metric": "auc",

    # regularization parameters
    "max_depth": 5,
    "subsample": 0.7,
    "colsample_bytree": 0.7,

    #lightgbm approach
    "tree_method": "hist"
}

results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=10)

[0]	train-auc:0.79231+0.00339	test-auc:0.79117+0.00668
[10]	train-auc:0.83662+0.00124	test-auc:0.83188+0.00309
[20]	train-auc:0.84091+0.00106	test-auc:0.83395+0.00242
[30]	train-auc:0.84285+0.00110	test-auc:0.83475+0.00223
[40]	train-auc:0.84403+0.00111	test-auc:0.83482+0.00224
[50]	train-auc:0.84596+0.00129	test-auc:0.83530+0.00243
[60]	train-auc:0.84757+0.00140	test-auc:0.83568+0.00245
[70]	train-auc:0.84930+0.00155	test-auc:0.83569+0.00248
[73]	train-auc:0.84978+0.00154	test-auc:0.83566+0.00240
CPU times: user 4.79 s, sys: 561 ms, total: 5.35 s
Wall time: 4.22 s


In [9]:
parameters = {
    #default
    "objective": "binary:logistic",
    "eta": 0.1,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": 1,
    "eval_metric": "auc",

    # regularization parameters
    "max_leaves": 32,
    "subsample": 0.7,
    "colsample_bytree": 0.7,

    #lightgbm approach
    "tree_method": "hist",
    "grow_policy": "lossguide"
}

results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=10,
                 folds=skf, verbose_eval=10)

[0]	train-auc:0.79231+0.00341	test-auc:0.79054+0.00672
[10]	train-auc:0.83985+0.00105	test-auc:0.83334+0.00266
[20]	train-auc:0.84434+0.00095	test-auc:0.83473+0.00222
[30]	train-auc:0.84622+0.00099	test-auc:0.83552+0.00216
[40]	train-auc:0.84787+0.00104	test-auc:0.83552+0.00206
[50]	train-auc:0.84996+0.00129	test-auc:0.83564+0.00218
[60]	train-auc:0.85198+0.00132	test-auc:0.83590+0.00201
[70]	train-auc:0.85414+0.00142	test-auc:0.83590+0.00210
[74]	train-auc:0.85509+0.00151	test-auc:0.83592+0.00195


In [10]:
parameters = {
    #default
    "objective": "binary:logistic",
    "eta": 0.01,
    "verbosity": 0,
    "nthread": 10,
    "random_seed": 1,
    "eval_metric": "auc",

    # regularization parameters
    "max_leaves": 32,
    "subsample": 0.7,
    "colsample_bytree": 0.7,

    #lightgbm approach
    "tree_method": "hist",
    "grow_policy": "lossguide"
}

results = xgb.cv(parameters, xgb_train, num_rounds, early_stopping_rounds=50,
                 folds=skf, verbose_eval=10)

[0]	train-auc:0.79231+0.00341	test-auc:0.79054+0.00672
[10]	train-auc:0.83784+0.00112	test-auc:0.83230+0.00288
[20]	train-auc:0.84031+0.00105	test-auc:0.83345+0.00236
[30]	train-auc:0.84039+0.00114	test-auc:0.83373+0.00209
[40]	train-auc:0.83986+0.00115	test-auc:0.83340+0.00196
[50]	train-auc:0.84061+0.00120	test-auc:0.83387+0.00205
[60]	train-auc:0.84085+0.00116	test-auc:0.83414+0.00211
[70]	train-auc:0.84149+0.00117	test-auc:0.83464+0.00213
[80]	train-auc:0.84177+0.00129	test-auc:0.83466+0.00207
[90]	train-auc:0.84194+0.00127	test-auc:0.83472+0.00213
[100]	train-auc:0.84221+0.00130	test-auc:0.83484+0.00220
[110]	train-auc:0.84254+0.00130	test-auc:0.83501+0.00220
[120]	train-auc:0.84284+0.00129	test-auc:0.83518+0.00230
[130]	train-auc:0.84272+0.00129	test-auc:0.83503+0.00228
[140]	train-auc:0.84302+0.00128	test-auc:0.83510+0.00229
[150]	train-auc:0.84316+0.00128	test-auc:0.83517+0.00235
[160]	train-auc:0.84328+0.00128	test-auc:0.83520+0.00238
[170]	train-auc:0.84347+0.00128	test-auc:0

### Some useful notes

1. Custom objective and evaluation functions – https://github.com/dmlc/xgboost/blob/master/demo/guide-python/custom_objective.py
2. Never use timeseries validation with **xgb.cv**, it is broken :(
3. Investigate your task and metrics. There are many objective functions that are worth to try.


# LightGBM

LightGBM is the new GBDT algorithm with **GOSS** and **EFB**.

### GOSS

**Gradient-based One-Side Sampling (GOSS).** While there is no native weight for data instance in GBDT, they notice that data instances with different gradients play different roles in the computation of information gain. In particular, according to the definition of information gain, those instances with larger gradients (i.e., under-trained instances) will contribute more to the information gain. Therefore, when down sampling the data instances, in order to retain the accuracy of information gain estimation, we should better keep those instances with large gradients (e.g., larger than a pre-defined threshold, or among the top percentiles), and only randomly drop those instances with small gradients. We prove that such a treatment can lead to a more accurate gain estimation than uniformly random sampling, with the same target sampling rate, especially when the value of information gain has a large range.

### EFB

**Exclusive Feature Bundling (EFB).** Usually in real applications, although there are a large number of features, the feature space is quite sparse, which provides us a possibility of designing a nearly lossless approach to reduce the number of effective features. Specifically, in a sparse feature space, many features are (almost) exclusive, i.e., they rarely take nonzero values simultaneously. Examples include the one-hot features (e.g., one-hot word representation in text mining). We can safely bundle such exclusive features. To this end, they design an efficient algorithm by reducing the optimal bundling problem to a graph coloring problem (by taking features as vertices and adding edges for every two features if they are not mutually exclusive), and solving it by a greedy algorithm with a constant approximation ratio.

LightGBM uses the histogram based algorithms, which bucketing continuous feature(attribute) values into discrete bins, to speed up training procedure and reduce memory usage. In 2017 it was implemented in Xgboost too.

### Tree Spliting

**Leaf-wise (Best-first) Tree Growth**

Most decision tree learning algorithms grow tree by level (depth)-wise, like the following image:

<img src='https://github.com/Microsoft/LightGBM/blob/master/docs/_static/images/level-wise.png?raw=true'   align='center'>

LightGBM grows tree by leaf-wise (best-first). It will choose the leaf with max delta loss to grow. When growing same #leaf, leaf-wise algorithm can reduce more loss than level-wise algorithm.

Leaf-wise may cause over-fitting when #data is small. So, LightGBM can use an additional parameter max_depth to limit depth of tree and avoid over-fitting (tree still grows by leaf-wise).

<img src='https://github.com/Microsoft/LightGBM/raw/master/docs/_static/images/leaf-wise.png'   align='center'>

### Histogram based

LightGBM uses the histogram based algorithms, which bucketing continuous feature(attribute) values into discrete bins, to speed up training procedure and reduce memory usage.

### Working with Categorical Features

We often convert the categorical features into one-hot coding. However, it is not a good solution in tree learner. The reason is, for the high cardinality categorical features, it will grow the very unbalance tree, and needs to grow very deep to achieve the good accuracy.

The basic idea is reordering the categories according to the relevance of training target. More specifically, reordering the histogram (of categorical feature) according to it's accumulate values (`sum_gradient` / `sum_hessian`), then find the best split on the sorted histogram.

On practice, you need to try both methods and choose the best for your data.


### Lightgbm Parameters Tunning


**Core parameters**

Begin with `learning_rate` parameter setting with default value <br>
`n_estimators`(number of boosting iterations) set 10k (some big number).<br>
`num_leaves` (number of leaves in one tree) set 155 as a starting point.<br>
`objective` –  the same as for xgboost `objective` (adding "regression_l1")

**Control Overfitting and Accuracy**

`max_depth` – max tree depth (`default=-1`). Start with tunning `num_leaves`, in the end try to change `max_depth`; <br>
`min_data_in_leaf` – very important parameter that helps to control overfitting (`default=20`);<br>
`colsample_bytree` – select part of features on each iteration (`default=1.0`). Always have to be tunned! <br>
`subsample` – select part of data without resampling (`default=1.0`). To enable it, set `subsample_freq` = 1 (other values always work worse)
`early_stopping_round` – as in xgboost



In [11]:
import lightgbm as lgb

parameters = {
    "objective": "binary",
    "learning_rate": 0.1,
    "num_threads": 10,
    "metric": "auc",
    "seed": 42
}
n_rounds = 10000

lgb_train = lgb.Dataset(X, label=y, free_raw_data=False)

In [12]:
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf, eval_train_metric=True, callbacks=[lgb.early_stopping(10), lgb.log_evaluation(period=10)])

[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004208 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.

In [13]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.1,
    "num_threads": 10,
    "metric": 'auc',
    "seed": 42,

    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1
}

result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf, eval_train_metric=True, callbacks=[lgb.early_stopping(10), lgb.log_evaluation(period=10)])

[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004621 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004452 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004073 seconds.
You can set `force_col_wise=true` to remove the 

In [14]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.1,
    "num_threads": 10,
    "metric": "auc",
    "seed": 42,

    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1,
    "min_data_in_leaf": 15
}

result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf, eval_train_metric=True, callbacks=[lgb.early_stopping(10), lgb.log_evaluation(period=10)])

[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005132 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002940 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002880 seconds.
You can set `force_row_wise=true` to remove the 

In [15]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.01,
    "num_threads": 10,
    "metric": "auc",
    "seed": 42,

    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1,
}


result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf, eval_train_metric=True, callbacks=[lgb.early_stopping(50), lgb.log_evaluation(period=100)])

[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004703 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003163 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004930 seconds.
You can set `force_col_wise=true` to remove the 

In [16]:
print(f"Train auc:      {result['train auc-mean'][-1]:.4f}, std: {result['train auc-stdv'][-1]:.4f}")
print(f"Validation auc: {result['valid auc-mean'][-1]:.4f}, std: {result['valid auc-stdv'][-1]:.4f}")



Train auc:      0.8590, std: 0.0015
Validation auc: 0.8368, std: 0.0025


In [17]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.01,
    "num_threads": 10,
    "metric": "auc",
    "seed": 42,

    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1,
    "min_data_in_leaf": 50
}


result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf, eval_train_metric=True, callbacks=[lgb.early_stopping(50), lgb.log_evaluation(period=100)])

[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002963 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004369 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003897 seconds.
You can set `force_col_wise=true` to remove the 

In [18]:
print(f"Train auc:      {result['train auc-mean'][-1]:.4f}, std: {result['train auc-stdv'][-1]:.4f}")
print(f"Validation auc: {result['valid auc-mean'][-1]:.4f}, std: {result['valid auc-stdv'][-1]:.4f}")



Train auc:      0.8564, std: 0.0013
Validation auc: 0.8369, std: 0.0024


In [19]:
X.NumberOfDependents.value_counts()

0.0     26626
1.0      8106
2.0      6071
3.0      3004
4.0       951
5.0       226
6.0        57
7.0        14
8.0         6
9.0         1
10.0        1
Name: NumberOfDependents, dtype: int64

In [20]:
parameters = {
    #default
    "objective": "binary",
    "learning_rate": 0.01,
    "num_threads": 10,
    "metric": "auc",
    "seed": 42,

    #regularization
    "colsample_bytree": 0.8,
    "subsample": 0.8,
    "subsample_freq": 1,
    "min_data_in_leaf": 15,

    #categorical features
    'cat_smooth': 10,
    'min_data_per_group': 50
}
lgb_train = lgb.Dataset(X, label=y, free_raw_data=False, categorical_feature=['NumberOfDependents'])
result = lgb.cv(parameters, lgb_train, n_rounds, folds=skf, eval_train_metric=True, callbacks=[lgb.early_stopping(50), lgb.log_evaluation(period=100)])

[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003965 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003127 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 642
[LightGBM] [Info] Number of data points in the train set: 30042, number of used features: 7
[LightGBM] [Info] Number of positive: 6684, number of negative: 23358
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002852 seconds.
You can set `force_row_wise=true` to remove the 

In [21]:
print(f"Train auc:      {result['train auc-mean'][-1]:.4f}, std: {result['train auc-stdv'][-1]:.4f}")
print(f"Validation auc: {result['valid auc-mean'][-1]:.4f}, std: {result['valid auc-stdv'][-1]:.4f}")



Train auc:      0.8591, std: 0.0014
Validation auc: 0.8367, std: 0.0025


# CatBoost

CatBoost is open-sourced machine learning algorithm from Yandex. It can easily integrate with deep learning frameworks like Google’s TensorFlow and Apple’s Core ML. It can work with diverse data types to help solve a wide range of problems that businesses face today. To top it up, it provides best-in-class accuracy.

“CatBoost” name comes from two words “Category” and “Boosting”.

### Quantization

Before learning, the possible values of objects are divided into disjoint ranges (buckets) delimited by the threshold values (splits). The size of the quantization (the number of splits) is determined by the starting parameters (separately for numerical features and numbers obtained as a result of converting categorical features into numerical features).

Quantization is also used to split the label values when working with categorical features. А random subset of the dataset is used for this purpose on large datasets.

Types: Median, Uniform, UniformAndQuantiles (Median + Uniform), MaxLogSum (maximize $ \large \sum_{i=1}^{n}log(weight)$, $\large n $ - number of unique objects, $\large weight $ – the number of times an object in the bucket is repeated), MinEntropy (minimize $ \large \sum_{i=1}^{n}weight \times log(weight)$), GreedyLogSum(maximize greedy).


### Dealing with Categorical Features

The main difference between Xgboost/LightGBM vs CatBoost is transforming categorical features.

Before each split is selected in the tree, categorical features are transformed to numeric features. This is done using various statistics on combinations of categorical features and combinations of categorical and numerical features.

The method of transforming categorical features to numerical features includes the following stages:
1. Permutating the set of input objects in a random order.
2. Converting the label value from a floating point to an integer. (for regression: "Quantization is performed on the label value. The mode and number of buckets ($ k + 1 $ ) are set in the starting parameters. All values are located inside a single bucket)
3. Transforming categorical features to numerical features.<br>
<br>

 $$ \Large ctr_i=\frac{CountInClass + prior}{TotalCount + 1} $$


 * Borders: <br>
**`countInClass`** is how many times the label value exceeded $ \large i $ for objects with the current categorical feature value. It is only counts objects that have been made in the order of the objects after shuffling. <br>
**`totalCount`** is the total number of objects (up to the current one) that have a feature value matching the current one.<br>
**`prior`** is defined by the starting parameters.<br>


 * Buckets:<br>
**`countInClass`** is how many times the label value was equal to $ \large i $ for objects with the current categorical feature value. It is only counts objects that have been made in the order of the objects after shuffling. <br>
**`totalCount`** is the total number of objects (up to the current one) that have a feature value matching the current one.<br>
**`prior`** is defined by the starting parameters.<br>
    
    
  * BinarizedTargetMeanValue:<br>
**`countInClass`** is the ratio of the sum of the label value integers for this categorical feature to the maximum label value integer.<br>
**`totalCount`** is the total number of objects that have a feature value matching the current one.<br>
**`prior`** is defined by the starting parameters.<br>


 * Counter<br>
$$ \Large ctr_i=\frac{CurCount + prior}{TotalCount + 1} $$
**`curCount`** is the total number of objects in the training dataset with the current categorical feature value.<br>
**`maxCount`** the number of objects in the training dataset with the most frequent feature value.<br>
**`prior`** is defined by the starting parameters.<br>

# Comparing Hyperparameters

<img src='https://cdn-images-1.medium.com/max/2000/1*A0b_ahXOrrijazzJengwYw.png' aligh='center'>

### CatBoost parameters tunning

**`loss_function`** – The metric to use in training (the same as objective in xgb/lgb)<br>
**`eval_metric`** – The metric used for overfitting detection (if enabled) and the best model selection (if enabled).<br>
**`iterations`** – The maximum number of trees<br>
**`learning_rate`** – Learing rate :) (default 0.03)<br>
**`random_seed`** – is not set by default. always set to reproduce results.<br>
**`subsample`** – Sample rate for bagging (default 0.66)<br>
**`use_best_model`** – True if a validation set is input (the eval_set parameter is defined) and at least one of the values of objects in this set. False otherwise<br>
**`depth`** – the same as max depth earlier<br>
**`rsm`** – Random subspace method. The percentage of features that can be used at each split selection. (colsample_bylevel)<br>
**`class_weights`** – Classes weights<br>
**`od_type`** – The type of the overfitting detector to use. (better to use Iter)<br>
**`od_wait`** – The number of iterations to continue the training after the metric value.<br>






In [23]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.3 MB/s eta 0:00:00


In [24]:
import catboost as ctb
parameters = {
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "iterations": 1000,
    "learning_rate": 0.03,
    "random_seed": 42,
    "od_wait": 30,
    "od_type": "Iter",
    "thread_count": 10
}

In [25]:
ctb_data = ctb.Pool(X,y)
result = ctb.cv(ctb_data, parameters, folds=skf, seed=42, verbose_eval=100, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/3]
0:	test: 0.7908426	best: 0.7908426 (0)	total: 72.2ms	remaining: 1m 12s
100:	test: 0.8354981	best: 0.8354981 (100)	total: 4.57s	remaining: 40.7s
200:	test: 0.8379776	best: 0.8379776 (200)	total: 9.38s	remaining: 37.3s
300:	test: 0.8389764	best: 0.8389764 (300)	total: 13.2s	remaining: 30.6s
400:	test: 0.8394747	best: 0.8395036 (387)	total: 19.1s	remaining: 28.5s
500:	test: 0.8396336	best: 0.8396418 (491)	total: 24.4s	remaining: 24.3s

bestTest = 0.8396896428
bestIteration = 540

Training on fold [1/3]
0:	test: 0.7807608	best: 0.7807608 (0)	total: 122ms	remaining: 2m 2s
100:	test: 0.8304067	best: 0.8304067 (100)	total: 5.76s	remaining: 51.3s
200:	test: 0.8339108	best: 0.8339117 (199)	total: 11.1s	remaining: 44.3s
300:	test: 0.8351095	best: 0.8351095 (300)	total: 13.8s	remaining: 32s
400:	test: 0.8359496	best: 0.8359496 (400)	total: 16.2s	remaining: 24.2s

bestTest = 0.8361638753
bestIteration = 451

Training on fold [2/3]
0:	test: 0.7746712	best: 0.7746712 (0)	total

In [26]:
result.loc[result["test-AUC-mean"] == result["test-AUC-mean"].max()]

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
540,540,0.833322,0.008113,0.390313,0.012081,0.377874,0.018799


In [28]:
len(X.columns)
categorical_features_indices = np.where(X.dtypes != np.float64)[0]

In [29]:
X.dtypes

age                                       int64
NumberOfTime30-59DaysPastDueNotWorse      int64
DebtRatio                               float64
NumberOfTimes90DaysLate                   int64
NumberOfTime60-89DaysPastDueNotWorse      int64
MonthlyIncome                           float64
NumberOfDependents                      float64
dtype: object

In [30]:
categorical_features_indices

array([0, 1, 3, 4])

In [31]:
parameters = {
    #default
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "iterations": 1000,
    "learning_rate": 0.03,
    "random_seed": 42,
    "od_wait": 50,
    "od_type": "Iter",
    "thread_count": 10
}

ctb_data = ctb.Pool(X,y,cat_features=categorical_features_indices)
result = ctb.cv(ctb_data, parameters, folds=skf, seed=42, verbose_eval=100, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/3]
0:	test: 0.8005543	best: 0.8005543 (0)	total: 149ms	remaining: 2m 28s
100:	test: 0.8327859	best: 0.8328213 (95)	total: 8.41s	remaining: 1m 14s
200:	test: 0.8354828	best: 0.8354828 (200)	total: 18.1s	remaining: 1m 12s
300:	test: 0.8365119	best: 0.8365119 (300)	total: 26.6s	remaining: 1m 1s
400:	test: 0.8374780	best: 0.8374780 (400)	total: 36.6s	remaining: 54.6s
500:	test: 0.8379505	best: 0.8379505 (500)	total: 46.7s	remaining: 46.5s
600:	test: 0.8382298	best: 0.8382517 (589)	total: 55.1s	remaining: 36.6s
700:	test: 0.8383296	best: 0.8383467 (682)	total: 1m 5s	remaining: 27.8s
800:	test: 0.8384636	best: 0.8384860 (767)	total: 1m 16s	remaining: 19s

bestTest = 0.8384860037
bestIteration = 767

Training on fold [1/3]
0:	test: 0.7864281	best: 0.7864281 (0)	total: 79.6ms	remaining: 1m 19s
100:	test: 0.8278314	best: 0.8278485 (97)	total: 7.89s	remaining: 1m 10s
200:	test: 0.8315115	best: 0.8315115 (200)	total: 17.1s	remaining: 1m 8s
300:	test: 0.8329617	best: 0.8329625 

In [32]:
result.loc[result["test-AUC-mean"] == result["test-AUC-mean"].max()]

,iterations,test-AUC-mean,test-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
995,995,0.835688,0.002791,0.385143,0.002266,0.362304,0.004327


# Summary


- Boosting algorithms are the best for heterogeneous data.
- Lightgbm is the fastest and usually the most accurate;
- CatBoost doesn't need a lot of tuning;
- CatBoost shows the best result on data with many categorical variables;
- Always try three methods and ensemble them;
- Remember about regularization and comparing your training score with validation;
- Use LightGBM for experiments and in the end execute other algorithms;
- Firstly, set default parameters, do feature engineering and then come back to parameters tuning;
- More practice will give you more understanding and intuition;
- Use magic of boosting in real life :)



# Sources:

- Introduction to Boosted Trees: http://xgboost.readthedocs.io/en/latest/model.html
- XGBoost, a Top Machine Learning Method on Kaggle, Explained : https://www.kdnuggets.com/2017/10/xgboost-top-machine-learning-method-kaggle-explained.html
- LightGBM: A Highly Efficient Gradient Boosting Decision Tree: https://papers.nips.cc/paper/6907-lightgbm-a-highly-efficient-gradient-boosting-decision-tree.pdf
- CatBoost: A machine learning library to handle categorical (CAT) data automatically: https://www.analyticsvidhya.com/blog/2017/08/catboost-automated-categorical-data/
- CatBoost: https://catboost.ai/docs/
- CatBoost tutorials: https://github.com/catboost/tutorials

### Homework:
* use the dataset and validation approach which you used for previous homework
* train xgboost, lightgbm, and catboost models and find the best hyperparameters for each algorithm
* compare results between them and previously trained random forest